In [12]:
import os
import requests
import operator
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.callbacks import get_openai_callback

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "RRL project"

os.environ["AZURE_OPENAI_API_KEY"] = "EhMIoJnOsNomEJ8TRfOEoIc1jC49AwdEwmZ8UDi4lh6dsUZ4WEArJQQJ99BAACYeBjFXJ3w3AAAAACOGl3Rt"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://week31004687013.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview"


### TODO:
- Add multihop question generation.
- Add features to solve failcase 2 and 3

In [13]:
def call_orchestrator(messages, model):
    "Takes in user query -> returns relevant tool to call from the list of available tools."
    
    template = """You are Hotel booking platform orchestrator who can use tools to retrieve relevant information.
        Your objective it to answer user query in the most optimal way and so you must use the tool available to
        to do so.
        
        Tools available: ['check_query', 'search_web', 'generate_answer', 'done']
        check_query: to check if most of the information to suggest a hotel is present in the query.
        search_web: to search the web with the user query for relevant hotels.
        generate_answer: to answer user query based on web search results.
        done: to break the workflow when the user query has been successfully answered.
        
        Return type: str
        return only the name of the tool to be used.
        
        History: {messages}"""
    
    print(messages)
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Orchestrator output: " + response.content)
    return response.content, cb.total_cost



def check_query(messages, model):
    """Takes in user query and checks if it has all relevant information to book hotel.
        yes -> return to the orchestrator. 
        no -> return to the user."""
    
    print(messages)
    template = """You are a query checker for hotel room booking agent. Your job is to verify if the given user query consist of all relevant information
        to proceed with suggesting a hotel for the user. The query must contain information such as number of days of stay, number of rooms, number of guests etc.
        
        Query: {messages}
        
        Return type: str
        return only True or False where True means the query is sufficient and False represents it is not."""
    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages[-2]),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Query check: " + response.content)
    return response.content, cb.total_cost


def get_updated_query(messages):
    "Ask the user to share more information about their booking choice."

    print(messages)
    updated_query = input("The entered information is not sufficient to search for hotels, please share more information like location, dates, number of guests, budget etc.")
    messages.append(updated_query)
    return updated_query

def search(messages):
    "Takes in user query -> searches the web -> return relevant information."
    print(messages)
    search_query = messages[-4]

    def fetch_full_text(url):
        try:
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
            response.raise_for_status()  # Raise an error for bad responses
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Extract main text content from <p> tags
            paragraphs = soup.find_all("p")
            full_text = "\n".join([p.get_text() for p in paragraphs])
            return full_text
        except requests.exceptions.RequestException as e:
            return f"Error fetching page: {e}"

    # Function to search DuckDuckGo
    def search_duckduckgo(query, num_results=3):
        searched_output = []
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=num_results))
        for i, search_results in enumerate(results, 1):
            link_text = fetch_full_text(search_results['href'])
            result = f"\nSearch: {i}\nTitle: {search_results['title']}\nBody: {link_text}"
            # print(result)
            searched_output.append(result)
        return searched_output
    
    search_results = search_duckduckgo(search_query, num_results=5)

    return search_results

def generate_answer(messages, query):
    "Based on the obtained search results answer the user query"

    print(messages)
    web_search = messages[-2]

    template = """You are a hotel booking platform QA bot employed to answer user query based on retrieved information. Do not use your own knowledge
        but rely only on the extracted information.
        
        Web search: {web_search}
        
        Query: {query}"""
    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "web_search": str(web_search),
                "query": str(query)
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Answer: " + response.content)
    return response.content, cb.total_cost

def done(messages):
    "Used as an indicator to terminate the program."
    
    if "done" in messages[-1]:
        return True


In [14]:
completed_task = False
messages = []
query = input("Hi, how can I assist you.") # hotels with 3 rooms 2 bathroom for 3 people for 3 nights
messages.append(query)

model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21",  temperature=0, max_tokens=512, timeout=None, max_retries=2)

while not completed_task:
    choice_tool, _ = call_orchestrator(messages, model)

    if choice_tool == "check_query":
        flag, _ = check_query(messages, model)
        if flag == "False":
            flag = False
        elif flag == "True":
            flag = True
        if not flag:
            query = get_updated_query(messages)
    elif choice_tool == "search_web":
        web_results = search(messages)
        messages.append(web_results)
        print(messages)
    elif choice_tool == "generate_answer":
        response = generate_answer(messages, query)
    elif choice_tool == "done":
        print("Inside ELIF")
        completed_task = done(messages)
        # print(completed_task)

["suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. the hotel have a more than 30 floors, at least 1 swimming pool, 2 rooms must include air conditioning and heating with a dressing room and bathroom.  make sure the room overlooks a beach. if you're able to find such hotel then reply with a yes, otherwise no and nothing else."]
Total Cost (USD): $0.000000
["suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. the hotel have a more than 30 floors, at least 1 swimming pool, 2 rooms must include air conditioning and heating with a dressing room and bathroom.  make sure the room overlooks a beach. if you're able to find such hotel then reply with a yes, otherwise no and nothing else.", 'Orchestrator output: check_query']
Total Cost (USD): $0.000000
["suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. the hotel have a more than 30 floors, at least 1 swimming pool, 

In [15]:
for i in messages:
    print(i)

suggest me hotels for a stay of three days for 4 guests between the budget of 900$ and 1200$. the hotel have a more than 30 floors, at least 1 swimming pool, 2 rooms must include air conditioning and heating with a dressing room and bathroom.  make sure the room overlooks a beach. if you're able to find such hotel then reply with a yes, otherwise no and nothing else.
Orchestrator output: check_query
Query check: True
Orchestrator output: search_web
["\nSearch: 1\nTitle: 17 of the Best Hotel Booking Sites to Book Cheaper Prices\nBody: Latest\nAirlines\nTravel News\nPoints & Card News\nBanks\nThe shortcut to finding your next travel card\nGet card recommendations in minutes. No personal info required.\nCredit Card Stories\nGuides\nReviews\nDeals\nMust Reads\nSave on your next trip with daily cheap travel news & tips delivered to your inbox.\nThanks for subscribing to The Daily Beat!\nSomething went wrong. Please try again.\nPopular Topics\n\n      Gunnar is a reporter and flight deal ana

In [16]:

print(messages[-2])

Answer: No


### Updates

In [ ]:
def call_orchestrator(messages, model):
    "Takes in user query -> returns relevant tool to call from the list of available tools."
    
    template = """You are Hotel booking platform orchestrator who can use tools to retrieve relevant information.
        Your objective it to answer user query in the most optimal way and so you must use the tool available to
        to do so.
        
        Tools available: ['check_query', 'search_web', 'generate_answer', 'done']
        check_query: to check if most of the information to suggest a hotel is present in the query.
        search_web: to search the web with the user query for relevant hotels.
        generate_answer: to answer user query based on web search results.
        done: to break the workflow when the user query has been successfully answered.
        
        Return type: str
        return only the name of the tool to be used.
        
        History: {messages}"""
    
    print(messages)
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Orchestrator output: " + response.content)
    return response.content, cb.total_cost



def check_query(messages, model):
    """Takes in user query and checks if it has all relevant information to book hotel.
        yes -> return to the orchestrator. 
        no -> return to the user."""
    
    print(messages)
    template = """You are a query checker for hotel room booking agent. Your job is to verify if the given user query consist of all relevant information
        to proceed with suggesting a hotel for the user. The query must contain information such as number of days of stay, number of rooms, number of guests etc.
        
        Query: {messages}
        
        Return type: str
        return only True or False where True means the query is sufficient and False represents it is not."""
    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "messages": str(messages[-2]),
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Query check: " + response.content)
    return response.content, cb.total_cost


def get_updated_query(messages):
    "Ask the user to share more information about their booking choice."

    print(messages)
    updated_query = input("The entered information is not sufficient to search for hotels, please share more information like location, dates, number of guests, budget etc.")
    messages.append(updated_query)
    return updated_query

def search(messages):
    "Takes in user query -> searches the web -> return relevant information."
    print(messages)
    search_query = messages[-4]

    def fetch_full_text(url):
        try:
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
            response.raise_for_status()  # Raise an error for bad responses
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Extract main text content from <p> tags
            paragraphs = soup.find_all("p")
            full_text = "\n".join([p.get_text() for p in paragraphs])
            return full_text
        except requests.exceptions.RequestException as e:
            return f"Error fetching page: {e}"

    # Function to search DuckDuckGo
    def search_duckduckgo(query, num_results=3):
        searched_output = []
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=num_results))
        for i, search_results in enumerate(results, 1):
            link_text = fetch_full_text(search_results['href'])
            result = f"\nSearch: {i}\nTitle: {search_results['title']}\nBody: {link_text}"
            # print(result)
            searched_output.append(result)
        return searched_output
    
    search_results = search_duckduckgo(search_query, num_results=5)

    return search_results

def generate_answer(messages, query):
    "Based on the obtained search results answer the user query"

    print(messages)
    web_search = messages[-2]

    template = """You are a hotel booking platform QA bot employed to answer user query based on retrieved information. Do not use your own knowledge
        but rely only on the extracted information.
        
        Web search: {web_search}
        
        Query: {query}"""
    
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    chain = prompt_perspectives | model
    with get_openai_callback() as cb:
        response = chain.invoke(
            {
                "web_search": str(web_search),
                "query": str(query)
            }
        )
        print(f"Total Cost (USD): ${format(cb.total_cost, '.6f')}")
    messages.append("Answer: " + response.content)
    return response.content, cb.total_cost

def done(messages):
    "Used as an indicator to terminate the program."
    
    if "done" in messages[-1]:
        return True


In [ ]:
completed_task = False
messages = []
query = input("Hi, how can I assist you.") # hotels with 3 rooms 2 bathroom for 3 people for 3 nights
messages.append(query)

model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21",  temperature=0, max_tokens=512, timeout=None, max_retries=2)

while not completed_task:
    choice_tool, _ = call_orchestrator(messages, model)

    if choice_tool == "check_query":
        flag, _ = check_query(messages, model)
        if flag == "False":
            flag = False
        elif flag == "True":
            flag = True
        if not flag:
            query = get_updated_query(messages)
    elif choice_tool == "search_web":
        web_results = search(messages)
        messages.append(web_results)
        print(messages)
    elif choice_tool == "generate_answer":
        response = generate_answer(messages, query)
    elif choice_tool == "done":
        print("Inside ELIF")
        completed_task = done(messages)
        # print(completed_task)